In [1]:
from gym import Env
from gym.spaces import Discrete, Tuple
import numpy as np
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

from py4j.java_gateway import JavaGateway

In [2]:
# Conecta no servidor de treinamento
gateway = JavaGateway()
minitruco_java = gateway.entry_point 

In [13]:
# Env customizado do gym

class MinitrucoEnv(Env):
    def __init__(self):
        # Ações que podemos tomar: jogar a carta 0, a carta 1 ou a carta 2
        self.action_space = Discrete(3)
        
        # Estados possíveis
        # - posições: 1=inferior, 2=direita, 3=superior, 4=esquerda    
        # - equipes: 1=posições 1 e 3; 2=posições 2 e 4
        # - cartas: podem valer -1 (null), 0 (fechada) ou um valor de 1
        #           a 14, conforme o valor relativo delas (cartas normais de 1
        #           a 10, manilhas de 11 a 14)
        # - rodadas: 1 a 3
        # - resultado da rodada: a equipe que venceu (1 ou 2), 3 para empate ou -1 para rodada não conlcuída
        # - booleanos (ex.: podeFechada) são 0 ou 1
        # TBD posJogadorPedindoAumento (acho que não zera depois do aumento)
        # TBD tento mineiro (talvez só varie as recompensas, mas é preciso especificar)
        # TBD baralho limpo (provavelmente só vamos excluir o range 1-4)
        self.observation_space = Tuple((
            Discrete(4, start=1),    # posJogador
            Discrete(2),             # baralhoSujo
            Discrete(2),             # podeFechada
            Discrete(3, start=1),    # numRodadaAtual
            Discrete(3, start=1),    # resultadoRodada1
            Discrete(3, start=1),    # resultadoRodada2
            Discrete(12, start=1),   # valorMao
            Discrete(13, start=0),   # valorProximaAposta
            Discrete(4, start=1),    # posJogadorPedindoAumento
            Discrete(4, start=1),    # posJogadorQueAbriuRodada
            Discrete(24, start=0),   # pontosEquipe1
            Discrete(24, start=0),   # pontosEquipe2
            Discrete(16, start=-1),  # cartaJogadaRodada1Pos1
            Discrete(16, start=-1),  # cartaJogadaRodada1Pos2
            Discrete(16, start=-1),  # cartaJogadaRodada1Pos3
            Discrete(16, start=-1),  # cartaJogadaRodada1Pos4
            Discrete(16, start=-1),  # cartaJogadaRodada2Pos1
            Discrete(16, start=-1),  # cartaJogadaRodada2Pos2
            Discrete(16, start=-1),  # cartaJogadaRodada2Pos3
            Discrete(16, start=-1),  # cartaJogadaRodada2Pos4
            Discrete(16, start=-1),  # cartaJogadaRodada3Pos1
            Discrete(16, start=-1),  # cartaJogadaRodada3Pos2
            Discrete(16, start=-1),  # cartaJogadaRodada3Pos3
            Discrete(16, start=-1),  # cartaJogadaRodada3Pos4
            Discrete(16, start=-1),  # carta1Jogador
            Discrete(16, start=-1),  # carta2Jogador
            Discrete(16, start=-1)   # carta3Jogador
        ))
        
        self.episodio = minitruco_java.novoEpisodio()
        self.state = self.episodio.estado().split()
        
    def step(self, action):
#         last_observation = self.current_observation
        self.episodio.executa(action)
        observation = self.episodio.estado().split()
        
        pontosEquipe1 = observation[10]
        pontosEquipe2 = observation[11]
        
#         pontosGanhosEquipe1 = observation[10] - last_observation[10]
#         pontosGanhoesEquipe2 = observation[11] - last_observation[11]
        
        reward = 0.0
        done = False
        if (pontosEquipe1 > 11):
            reward += 1.0
            done = True
        elif (pontosEquipe2 > 11):
            reward += -1.0
            done = True
        
        return observation, reward, done, {}

    def render(self):
        pass
    
    def reset(self):
        self.episodio.finaliza()
        
        self.episodio = minitruco_java.novoEpisodio()
        return self.episodio.estado().split()
    

In [4]:
###### Testando a comunicação com o servidor de treino

In [5]:
gateway = JavaGateway()
minitruco_java = gateway.entry_point 

In [6]:
episodio = minitruco_java.novoEpisodio()
print(episodio.estado().split())

['1', '1', '0', '1', '-1', '-1', '1', '3', '0', '1', '0', '0', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '-1', '8', '12', '5']


In [7]:
episodio.executa(0)
print(episodio.estado().split())

['1', '1', '1', '2', '2', '-1', '1', '3', '0', '2', '0', '0', '8', '13', '4', '3', '-1', '4', '8', '9', '-1', '-1', '-1', '-1', '12', '5', '-1']


In [8]:
episodio.finaliza()

In [9]:
#################################

In [14]:
env = MinitrucoEnv()